<a href="https://colab.research.google.com/github/rp-bot/audio-sentiment-analysis/blob/main/audio_sentiment_analysis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>